分布式admm算法：
$$ \min_{W}\sum_{i=1}^{n} L_{i}(x_{i};W) $$
等价于
$$ \min_{W_{1} \cdots W_{n},Z} \sum_{i=1}^{n} L_{i}(x_{i};W_{i}) $$
$$ s.t. \quad W_{i} = Z \quad (i = 1,\cdots , n) $$
拉格朗日函数为：
$$ L_{c} = \sum_{i=1}^{n}L_{i}(x_{i};W_{i}) + \sum_{i=1}^{n} V_{i}\odot 
(W_{i}-Z_{i}) + \frac{c}{2} \sum_{i=1}^{n}\left \| W_{i} -Z \right\|_{F}^{2} $$
迭代公式
$$1.\quad W_{i}^{k+1} = arg\min_{W_{i}} (L_{i}(x_{i};W_{i}) + V_{i}^{k}\odot (W_{i}-Z_{i}^{k}) + \frac{c}{2}\left \| W_{i} - Z^{k} \right \|_{F}^{2})$$
$$\quad\qquad= arg\min_{W_{i}} (L_{i}(x_{i};W_{i}) +  \frac{c}{2}\left \| W_{i} - Z^{k}  + \frac{V_{i}^{k}}{c} \right \|_{F}^{2})\quad (\forall i) $$
$$2.\quad  Z^{k+1} = arg \min_{Z}\frac{c}{2} \sum_{i=1}^{n} \left \|Z -  W_{i}^{k+1} - \frac{V_{i}^{k}}{c} \right\|_{F}^{2} $$
$$= \frac{1}{n}\sum_{i=1}^{n}( W_{i}^{k+1} + \frac{V_{i}^{k}}{c}) $$
$$3.V_{i}^{k+1} = V_{i}^{k} + c(W_{i}^{k} - Z^{k+1}) \qquad (\forall i)$$

上面的admm算法与nn相结合就有：

1. 其中迭代公式$1.$中$W_{k+1}$的更新需要用梯度下降的方式
2. to be continue...

在test_cnn_admm.ipynb中admm算法实现是有问题的，每次迭代过程$L_{i}(x_{i},W_{i})$由于输入$x_{i}$的变化改变了，相当于admm同步的过程只是走了一步，相当于是实现了“一步”admm算法，其实也是work的，但是不是最原滋原味的admm。